* Pandas が、RDB にアクセスするごとに、「dbapi2 には対応しているけど sqlite 以外はちゃんとテストしてないよ」という warning を出すので、warning の出力を抑止する。(必要があれば、コメントアウトして実行する)

```
UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
```

In [1]:
import warnings

#warnings.filterwarnings('default')
#warnings.filterwarnings('error')
warnings.filterwarnings('ignore')
#warnings.filterwarnings('always')
#warnings.filterwarnings('module')
#warnings.filterwarnings('once')

# データベースへの接続

In [2]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

connection_config = {
    'user': 'root',
    'password': 'root',
    'host': 'postgres',
    'port': '5432',
    'database': 'testdb'
}

def get_connection():
    # ** は、オブジェクトを引数に展開する
    return psycopg2.connect(**connection_config)

def get_engine():
    return create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**connection_config))

# データの読み込み

* With 文で、connection を開くことによって、自動的に close する。例外が起きなければ commit もしてくれる

In [3]:
with get_connection() as connection:
    df = pd.read_sql(sql = 'SELECT * FROM iris_name;', con = connection, index_col = ["id"])

df

,cd,name
id,,
1,0,setosa
2,1,versicolor
3,2,virginica


# データの書き込み

* With 文で、engine を開くことによって、自動的に close する
* Pandas のデフォルト設定では、sqlite が前提。Postgresql で使う場合には、sqlalchemy の engine を入れ替える必要がある。
* to_sql() のオプション
    * if_exists
        * fail : エラーにする
        * append : テーブルにデータを追加する
        * replace : 強制定期に DataFrame のテーブルを作る
    * index
        * True : DataFraem の行番号を ${index_label} 列に格納する。index_label 未指定時は、列名には "index" が指定される
        * False : index を考慮しない
    * INSERT が遅い対策。(method = 'multi' 指定)
    * 大量データでエラー対策 (chunksize=5000 で 5000行単位で処理する)

# テーブルの作り方

* いったん、if_exists='replace', index=False で、テーブルを作る (CREATE TABLE)
* RDB のメンテナンスツール (pgadmin4) をつかって、テーブルに id 列 (bigserial) を作る。さっき pandas から入れたデータには id がふられる    
* 以降は、if_exists='append', index=False で実行する (つまり、to_sql からは id 列を関知しないようにする)


In [4]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')
print(iris.head())
with get_engine().connect() as connection:
    iris.to_sql('iris', con=connection, if_exists='replace', index=False, method='multi', chunksize=5000)

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [6]:
with get_connection() as connection:
    df = pd.read_sql(sql = 'SELECT * FROM iris;', con = connection, index_col = ["id"])

df

,sepal_length,sepal_width,petal_length,petal_width,species
id,,,,,
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,virginica
147,6.3,2.5,5.0,1.9,virginica
148,6.5,3.0,5.2,2.0,virginica


# 2022 時点での Pandas では、Update Delete はできない

* psycopg2 で、SQL 文を発行するしかない。
* Upsert (Update/Insert) 機能の Pull Request が動いているので、そのうち実装されるだろう https://github.com/pandas-dev/pandas/pull/29636
* まぁ、AI の学習データと学習結果を置いておくだけなんで、Create Read だけでいいかな・・・とりあえず

In [7]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute("DELETE FROM IRIS WHERE ID=1")
        cursor.execute("UPDATE IRIS SET sepal_length = 999.9 WHERE ID=2")
    
with get_connection() as connection:
    df = pd.read_sql(sql = 'SELECT * FROM IRIS', con = connection, index_col = ["id"])

df

,sepal_length,sepal_width,petal_length,petal_width,species
id,,,,,
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
...,...,...,...,...,...
147,6.3,2.5,5.0,1.9,virginica
148,6.5,3.0,5.2,2.0,virginica
149,6.2,3.4,5.4,2.3,virginica


# 生の psycopg2 で SELECT 文を発行するのは難しいので、Pandas 使ったほうがいい
* cursor.description で、列名を取得できるので、cursor.fetchall() で帰ってくるタプルと組み合わせて Pandas.DataFrame にできる
* 最初っからread_sqlで良くね?
* なにかの時系列データで、持っているデータの最後のデータを取るやり方の練習 (取ったもの以降を取得して格納する)

In [21]:
with get_connection() as connection:
    with connection.cursor() as cursor:
        cursor.execute('SELECT MAX(sepal_length) AS MAX_sepal_length, species FROM IRIS GROUP BY species')
        df = pd.DataFrame(cursor.fetchall(), columns=[col.name for col in cursor.description])

df

,max_sepal_length,species
0,999.9,setosa
1,7.9,virginica
2,7.0,versicolor


In [26]:
with get_connection() as connection:
    df = pd.read_sql(sql = 'SELECT * FROM IRIS ORDER BY petal_width DESC LIMIT 1', con = connection, index_col = ["id"])

df

,sepal_length,sepal_width,petal_length,petal_width,species
id,,,,,
101,6.3,3.3,6.0,2.5,virginica
